In [7]:
import os
import openai
from dotenv import load_dotenv
import os


In [9]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
from llama_index import SimpleDirectoryReader


In [4]:
documents = SimpleDirectoryReader(
    input_files=['../data/Raptor Contract.docx']
).load_data()

In [5]:
print(type(documents),'\n')
print(len(documents),'\n')
print(type(documents[0]),'\n')
print(documents[0])

<class 'list'> 

1 

<class 'llama_index.schema.Document'> 

Doc ID: 513d21d9-b56f-4d63-9109-e2cfbcb37598
Text: [R&G Draft 12.__.2021]
STOCK PURCHASE AGREEMENT  BY AND AMONG  [BUYER],  [TARGET COMPANY],
THE SELLERS LISTED ON SCHEDULE I HERETO  AND  THE SELLERS’
REPRESENTATIVE NAMED HEREIN  Dated as of [●]    [This document is
intended solely to facilitate discussions among the parties identified
herein.  Neither this document nor such di...


## RAG pipeline

In [6]:
from llama_index import Document
document = Document(text='\n\n'.join([doc.text for doc in documents]))

In [8]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

In [9]:
llm = OpenAI(model='gpt-3.5-turbo', temperature=0.1)
service_context = ServiceContext.from_defaults(
  llm=llm,
  embed_model='local:BAAI/bge-small-en-v1.5'
)
index = VectorStoreIndex.from_documents([document],service_context=service_context)

In [10]:
query_engine = index.as_query_engine()

In [11]:
response = query_engine.query(
  'Are there any conditions to the closing??'
)
print(str(response))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Yes, there are conditions to the closing as outlined in the agreement. These conditions include the delivery of various documents and certificates, written resignations of certain officers and directors, payoff and lien release letters, cancellation acknowledgements for vested options and warrants, electronic copies of documentation, execution of agreements, and compliance with certain Treasury Regulations.


## Evaluation  using TruLens

In [12]:

eval_questions = []
with open('/Users/Aaron/ContractAdvisorRAG/data/eval_questions.txt', 'r') as file:
    for line in file:
        item = line.strip()
        print(item)
        eval_questions.append(item)

How much is the escrow amount?
Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?
What is the purpose of the escrow?
Are there any conditions to the closing?


In [13]:

new_question = "Whose consent is required for the assignment of the Agreement by the Buyer?"
eval_questions.append(new_question)

In [14]:
print(eval_questions)

['How much is the escrow amount?', 'Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?', 'What is the purpose of the escrow?', 'Are there any conditions to the closing?', 'Whose consent is required for the assignment of the Agreement by the Buyer?']


In [1]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

TypeError: type 'typing.TypeVar' is not an acceptable base type

In [2]:
!pip uninstall trulens_eval

Found existing installation: trulens-eval 0.14.0
Uninstalling trulens-eval-0.14.0:
  Would remove:
    /Users/Aaron/.pyenv/versions/3.12.0/envs/rag/bin/trulens-eval
    /Users/Aaron/.pyenv/versions/3.12.0/envs/rag/lib/python3.12/site-packages/trulens_eval-0.14.0.dist-info/*
    /Users/Aaron/.pyenv/versions/3.12.0/envs/rag/lib/python3.12/site-packages/trulens_eval/*
Proceed (Y/n)? 